# 50인 데이터 매칭 알고리즘

사용 언어: Python

가치관 질문은 사용하지 않는다고 가정하고 우선 제외한 상태로 작성했습니다.

현재 구현 버전에서는 성별/연령 차이를 구현하지 않았습니다.

(09/09 수정: 매칭 알고리즘 보완)

(09/15 수정: 매칭 알고리즘 재수정, 2인씩 매칭하도록 고침. 전체 응답차 내에서 순위를 정해서 내려오는 방식으로 수정, 성별 및 연령 도입)

## 데이터 정제

데이터 분석 및 정리에 필요한 라이브러리(pandas, numpy)를 불러옵니다.

주어진 엑셀 파일에서 미리 사용하지 않을 열(타임스탬프, 가치관 질문, 빈 열 등)을 삭제하고, CSV 파일형식으로 변환하여 불러옵니다.
dataCount 변수에 현재 데이터 개수(응답 인원수)를 저장합니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# set of column names
headerNames = ["q1", "q2", "q3", "q4", "q5", "q6", "q7","q8", "q9", "q10", "gender", "age", "polAlign"]

# read csv-converted data from excel file
# current data is cleaned to exclude empty and/or meaningless columns
answerData = pd.read_csv('data/50data_answeronly.csv', header =0, names=headerNames)

# save number of entries
dataCount = len(answerData.index)

비수치화 되어 있는 응답 결과를 숫자 형태의 데이터값으로 변환해 줍니다.

In [3]:
# replace all redundant text to interger values

answerData.replace(["① 매우 찬성","② 찬성", "③ 중립", "④ 반대", "⑤ 매우 반대", "남", "여"], [1, 2, 3, 4, 5, "M", "F"], inplace = True)

## 응답 유사도 측정

모든 개인과 개인 간의 응답 유사도를 저장할 (데이터 수)^2 사이즈의 행렬인 diffMatrix를 새로 만듭니다.
기본적으로는 NaN (값이 없는 상태)로 초기화합니다.

In [4]:
# create a new dataframe to save values
# this will be very time and cost extensive, might be viable to change this part
# will be durable for several hundred calculations though...?

diffMatrix = pd.DataFrame(np.nan, index=range(dataCount), columns=range(dataCount), dtype='float')

각 개인과 개인의 응답 결과를 비교하여, 각 질문에 대한 총 응답 차이값 = Sum(|개별 응답 차|) 값을 저장합니다.
i번째 응답자와 j번째 응답자의 응답 차이값은 diffMatrix의 i행 j열에 저장됩니다. (j행 i열의 값과 동일합니다.)
자기 자신과의 응답차는 계산하지 않도록 짜 두어, 대각선으로 NaN 값이 나옵니다.

(50인 데이터에서는 수 초 내에 결과가 나오지만, 실제 데이터에서는 응답차 값을 계산하는 것에 상당한 시간이 소요될 것으로 예상됩니다.
 현재 모델에서는 개인과 개인 간의 총 응답차 값을 모두 계산해야 하기 때문에 이 부분이 필수적이라는 점 또한 유의해 주시기 바랍니다.)
 
(09/15 수정: 연령 및 성별 차이를 반영하였습니다. 응답차가 최우선적으로 반영되지만, 성별/연령이 순차적으로 반영될 수 있게 성별은 0.1값, 연령은 0.001값으로 계산합니다.)

In [5]:
for i in range(dataCount):
    for j in range(i, dataCount):
        tempVal=0;
        if (i==j):
            diffMatrix.iloc[i][j] = np.nan
            continue
        for k in range(10):
            # iterates through 10 for the number of questions in the dataframe
            tempVal += answerData.iloc[i][k]-answerData.iloc[j][k]
            
        tempVal = abs(tempVal)
        # assign values for gender/age difference
        if answerData.iloc[i]["gender"] != answerData.iloc[j]["gender"]:
            tempVal += 0.1
            
        tempVal += abs(answerData.iloc[i]["age"]-answerData.iloc[j]["age"])*0.001
        
        # Assign same value to the flipped index to save computation time
        diffMatrix.iloc[i][j] = tempVal
        diffMatrix.iloc[j][i] = tempVal

diffMatrix 내에 저장되어 있는 모든 값들의 중간값을 diffMedian 변수에 저장합니다.
50인 테스트 데이터셋의 경우, 중간값은 5.111에서 형성되었습니다.

In [6]:
# Calculate the median value of all the diff values (currently, it's 5)
diffMedian = diffMatrix.stack().median()

print(diffMedian)

5.111


## 매칭 알고리즘 - 전체 매칭: 1인 기준 (09/15 추가) 

위와 동일한 결과를 기준으로, 같은생각/다른생각에 대하여 각 1인씩만 매칭하는 알고리즘으로 변형시켜 보았습니다.

(앞선 과정과 유사하거나 같은 기능을 하는 변수들은 _all_1 을 붙여서 구분합니다)

In [7]:
samePairs_all_1 = []

# Save the number of times each index has been matched
matchSame_all_1 = [0]*dataCount

# Copy the entire diffMatrix values
sameCount_matrix_1 = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    sameCount_matrix_1[i][i] = 100

while np.min(sameCount_matrix_1) < diffMedian and np.sum(matchSame_all_1) < dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmin(sameCount_matrix_1), (dataCount, dataCount))
    
    if matchSame_all_1[tempInd[0]] >= 1 or matchSame_all_1[tempInd[1]] >= 1:
        sameCount_matrix_1[tempInd[0],tempInd[1]] = 100
        continue
    
    tempRank_0 = (sameCount_matrix_1[tempInd[0]] == 100).sum() 
    tempRank_1 = (sameCount_matrix_1[tempInd[1]] == 100).sum() 
    
    samePairs_all_1.append([tempInd[0], tempInd[1], tempRank_0, tempRank_1])
    sameCount_matrix_1[tempInd[0],tempInd[1]] = 100
    sameCount_matrix_1[tempInd[1],tempInd[0]] = 100
    
    # increment counts 
    matchSame_all_1[tempInd[0]] += 1
    matchSame_all_1[tempInd[1]] += 1
    
            
print(samePairs_all_1)

[[1, 23, 1, 1], [27, 44, 1, 1], [25, 48, 1, 1], [17, 21, 1, 1], [24, 40, 1, 1], [8, 12, 1, 1], [13, 16, 1, 1], [15, 32, 1, 1], [18, 33, 1, 1], [26, 47, 1, 1], [0, 7, 1, 1], [4, 5, 1, 1], [9, 20, 1, 1], [2, 31, 1, 1], [37, 39, 1, 1], [36, 45, 3, 1], [28, 29, 3, 3], [11, 43, 1, 3], [10, 14, 5, 5], [3, 49, 2, 1], [35, 46, 15, 14], [34, 38, 3, 1], [19, 22, 15, 5], [41, 42, 1, 2]]


다른생각 매칭 또한 같은 원리로 진행했습니다.

In [8]:
diffPairs_all_1 = []

# Save the number of times each index has been matched
matchDiff_all_1 = [0]*dataCount

# Copy the entire diffMatrix values
diffCount_matrix_1 = diffMatrix.to_numpy(copy = True)

for i in range(dataCount):
    diffCount_matrix_1[i][i] = -100

while np.max(diffCount_matrix_1) > diffMedian and np.sum(matchDiff_all_1) < 1*dataCount:
    # calculate the indices of the current minimun value
    tempInd = np.unravel_index(np.argmax(diffCount_matrix_1), (dataCount, dataCount))
    
    if matchDiff_all_1[tempInd[0]] >= 1 or matchDiff_all_1[tempInd[1]] >= 1:
        diffCount_matrix_1[tempInd[0],tempInd[1]] = -100
        continue
    
    tempRank_0 = (diffCount_matrix_1[tempInd[0]] == -100).sum() 
    tempRank_1 = (diffCount_matrix_1[tempInd[1]] == -100).sum() 
    
    diffPairs_all_1.append([tempInd[0], tempInd[1], tempRank_0, tempRank_1])
    diffCount_matrix_1[tempInd[0],tempInd[1]] = -100
    diffCount_matrix_1[tempInd[1],tempInd[0]] = -100
    
    # increment counts 
    matchDiff_all_1[tempInd[0]] += 1
    matchDiff_all_1[tempInd[1]] += 1
    
            
print(diffPairs_all_1)

[[30, 41, 1, 1], [25, 42, 2, 2], [38, 48, 3, 3], [22, 34, 4, 4], [16, 47, 5, 5], [13, 26, 6, 6], [19, 49, 7, 8], [3, 27, 10, 9], [8, 14, 10, 10], [10, 46, 10, 10], [7, 12, 11, 11], [9, 44, 11, 13], [0, 20, 16, 16], [15, 37, 18, 18], [11, 28, 18, 18], [2, 43, 18, 18], [32, 39, 19, 19]]


In [10]:
np.savetxt("samepairs_1.csv", samePairs_all_1, delimiter=",", fmt="%s")
np.savetxt("diffPairs_1.csv", diffPairs_all_1, delimiter=",", fmt="%s")

매칭 결과를 확인하기 위해 표로 정리했습니다. 50인 데이터를 기준으로, 같은생각/다른생각 모두 매칭되지 않은 경우가 1명 존재합니다.

중간값 등의 도입으로 인해, 같은생각/다른생각 둘 중 한 가지 이상이 매칭되지 않는 경우가 다수 존재합니다.

In [ ]:
matchData_1=pd.DataFrame(
    {'같은생각 매칭 수':matchSame_all_1, '다른생각 매칭 수':matchDiff_all_1, '합':np.add(matchSame_all_1,matchDiff_all_1)})

matchData_1